### Query an Index Deployed to Vertex Matching Engine
**NOTE** This notebook can only be run after completing `matching_engine_ann.ipynb`. You must run this notebook in the same VPC network as your deployed index endpoint. 

In [ ]:
!pip install grpcio-tools

Restart Kernel

#### Setup

In [1]:
import grpc
import tensorflow_hub as hub
from google.cloud import aiplatform_v1

PROJECT = !(gcloud config get-value core/project)
PROJECT = PROJECT[0]
REGION = "us-central1"
PARENT = f"projects/{PROJECT}/locations/{REGION}"
ENDPOINT = f"{REGION}-aiplatform.googleapis.com"

#### Online Queries
Now you can query against the deployed index through the online querying gRPC API (Match Service). You need to generate the client from protocol buffer definition.

In [ ]:
%%writefile match_service.proto

syntax = "proto3";

package google.cloud.aiplatform.container.v1beta1;

import "google/rpc/status.proto";

// MatchService is a Google managed service for efficient vector similarity
// search at scale.
service MatchService {
  // Returns the nearest neighbors for the query. If it is a sharded
  // deployment, calls the other shards and aggregates the responses.
  rpc Match(MatchRequest) returns (MatchResponse) {}

  // Returns the nearest neighbors for batch queries. If it is a sharded
  // deployment, calls the other shards and aggregates the responses.
  rpc BatchMatch(BatchMatchRequest) returns (BatchMatchResponse) {}
}

// Parameters for a match query.
message MatchRequest {
  // The ID of the DeploydIndex that will serve the request.
  // This MatchRequest is sent to a specific IndexEndpoint of the Control API,
  // as per the IndexEndpoint.network. That IndexEndpoint also has
  // IndexEndpoint.deployed_indexes, and each such index has an
  // DeployedIndex.id field.
  // The value of the field below must equal one of the DeployedIndex.id
  // fields of the IndexEndpoint that is being called for this request.
  string deployed_index_id = 1;

  // The embedding values.
  repeated float float_val = 2;

  // The number of nearest neighbors to be retrieved from database for
  // each query. If not set, will use the default from
  // the service configuration.
  int32 num_neighbors = 3;

  // The list of restricts.
  repeated Namespace restricts = 4;

  // Crowding is a constraint on a neighbor list produced by nearest neighbor
  // search requiring that no more than some value k' of the k neighbors
  // returned have the same value of crowding_attribute.
  // It's used for improving result diversity.
  // This field is the maximum number of matches with the same crowding tag.
  int32 per_crowding_attribute_num_neighbors = 5;

  // The number of neighbors to find via approximate search before
  // exact reordering is performed. If not set, the default value from scam
  // config is used; if set, this value must be > 0.
  int32 approx_num_neighbors = 6;

  // The fraction of the number of leaves to search, set at query time allows
  // user to tune search performance. This value increase result in both search
  // accuracy and latency increase. The value should be between 0.0 and 1.0. If
  // not set or set to 0.0, query uses the default value specified in
  // NearestNeighborSearchConfig.TreeAHConfig.leaf_nodes_to_search_percent.
  int32 leaf_nodes_to_search_percent_override = 7;
}

// Response of a match query.
message MatchResponse {
  message Neighbor {
    // The ids of the matches.
    string id = 1;

    // The distances of the matches.
    double distance = 2;
  }
  // All its neighbors.
  repeated Neighbor neighbor = 1;
}

// Parameters for a batch match query.
message BatchMatchRequest {
  // Batched requests against one index.
  message BatchMatchRequestPerIndex {
    // The ID of the DeploydIndex that will serve the request.
    string deployed_index_id = 1;

    // The requests against the index identified by the above deployed_index_id.
    repeated MatchRequest requests = 2;

    // Selects the optimal batch size to use for low-level batching. Queries
    // within each low level batch are executed sequentially while low level
    // batches are executed in parallel.
    // This field is optional, defaults to 0 if not set. A non-positive number
    // disables low level batching, i.e. all queries are executed sequentially.
    int32 low_level_batch_size = 3;
  }

  // The batch requests grouped by indexes.
  repeated BatchMatchRequestPerIndex requests = 1;
}

// Response of a batch match query.
message BatchMatchResponse {
  // Batched responses for one index.
  message BatchMatchResponsePerIndex {
    // The ID of the DeployedIndex that produced the responses.
    string deployed_index_id = 1;

    // The match responses produced by the index identified by the above
    // deployed_index_id. This field is set only when the query against that
    // index succeed.
    repeated MatchResponse responses = 2;

    // The status of response for the batch query identified by the above
    // deployed_index_id.
    google.rpc.Status status = 3;
  }

  // The batched responses grouped by indexes.
  repeated BatchMatchResponsePerIndex responses = 1;
}

// Namespace specifies the rules for determining the datapoints that are
// eligible for each matching query, overall query is an AND across namespaces.
message Namespace {
  // The string name of the namespace that this proto is specifying,
  // such as "color", "shape", "geo", or "tags".
  string name = 1;

  // The allowed tokens in the namespace.
  repeated string allow_tokens = 2;

  // The denied tokens in the namespace.
  // The denied tokens have exactly the same format as the token fields, but
  // represents a negation. When a token is denied, then matches will be
  // excluded whenever the other datapoint has that token.
  //
  // For example, if a query specifies {color: red, blue, !purple}, then that
  // query will match datapoints that are red or blue, but if those points are
  // also purple, then they will be excluded even if they are red/blue.
  repeated string deny_tokens = 3;
}

Third party dependencies

In [ ]:
!mkdir third_party && \
    cd third_party && \
    git clone https://github.com/googleapis/googleapis.git

Compile the protocol buffer, and then `match_service_pb2.py` and `match_service_pb2_grpc.py` are generated.

In [ ]:
!python -m grpc_tools.protoc -I=. --proto_path=third_party/googleapis --python_out=. --grpc_python_out=. match_service.proto

Get the IP of the deployed index endpoint

In [2]:
index_endpoint_client = aiplatform_v1.IndexEndpointServiceClient(
    client_options=dict(api_endpoint=ENDPOINT)
)

DEPLOYED_INDEX_SERVER_IP = (
    list(index_endpoint_client.list_index_endpoints(parent=PARENT))[0]
    .deployed_indexes[0]
    .private_endpoints.match_grpc_address
)
DEPLOYED_INDEX_SERVER_IP

'10.64.0.5'

In [3]:
import match_service_pb2
import match_service_pb2_grpc

channel = grpc.insecure_channel(f"{DEPLOYED_INDEX_SERVER_IP}:10000")
stub = match_service_pb2_grpc.MatchServiceStub(channel)

Create functions to 
* Take a sentence and convert it to a vector
* Take a vector and send a request to Vertex Matching Engine

In [4]:
import match_service_pb2
import match_service_pb2_grpc
import tensorflow_hub as hub

channel = grpc.insecure_channel(f"{DEPLOYED_INDEX_SERVER_IP}:10000")
stub = match_service_pb2_grpc.MatchServiceStub(channel)

embed = hub.load("https://tfhub.dev/google/universal-sentence-encoder/4")


def sentence_to_vector(s):
    embeddings = embed([s])
    return embeddings[0].numpy().tolist()


def get_ann(vector, num_neighbors=5):
    request = match_service_pb2.MatchRequest(num_neighbors=num_neighbors)
    request.deployed_index_id = "deployed-movie-index"
    for val in vector:
        request.float_val.append(val)

    response = stub.Match(request)
    return response

2022-06-29 15:21:04.395240: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.
2022-06-29 15:21:08.387966: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


Create an example movie overview (feel free to play around and write your own) to see what movies are similar based off their overview embeddings) and send it through Google's Universal Sentence Encoder to generate a vector.

In [5]:
s = """
In this classic thriller, Kyle pushes a machine learning algorithm 
to production without thinking twice. The events that follow change
how he views artifical intelligence forever. 
"""

vector = sentence_to_vector(s)

Make a request with the vector to Matching Engine. Remember in our index metadata we specified `DOT_PRODUCT_DISTANCE`. This will return a number between 0 and 1. The larger the number the greater the similarity.  

In [7]:
get_ann(vector, num_neighbors=15)

neighbor {
  id: "Evolver"
  distance: 0.5481205582618713
}
neighbor {
  id: "The Questor Tapes"
  distance: 0.500476598739624
}
neighbor {
  id: "D.A.R.Y.L."
  distance: 0.4930911064147949
}
neighbor {
  id: "Source Code"
  distance: 0.48311907052993774
}
neighbor {
  id: "A.I. Artificial Intelligence"
  distance: 0.4776439070701599
}
neighbor {
  id: "Justice Is Mind"
  distance: 0.4682193994522095
}
neighbor {
  id: "Animal"
  distance: 0.46664437651634216
}
neighbor {
  id: "Saturn 3"
  distance: 0.4630534052848816
}
neighbor {
  id: "Flesh Wounds"
  distance: 0.4608614444732666
}
neighbor {
  id: "A.P.E.X."
  distance: 0.4607503414154053
}
neighbor {
  id: "Terminator Genisys"
  distance: 0.45924970507621765
}
neighbor {
  id: "Hologram Man"
  distance: 0.45832666754722595
}
neighbor {
  id: "DxM"
  distance: 0.4581947922706604
}
neighbor {
  id: "Prototype"
  distance: 0.45451611280441284
}
neighbor {
  id: "Debug"
  distance: 0.45419520139694214
}